In [0]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
  drive.mount('/content/gdrive')
  print('Google Drive is mounted\n')
else:
  print('Google Drive is already mounted\n')

In [0]:
import numpy as np
import random
# 랜덤시드 고정시키기
np.random.seed(777)

from keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range=[0.8, 2.0],
                                  horizontal_flip=True,
                                  vertical_flip=True,                  # data augmentation 출처 https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/
                                  fill_mode='nearest'
                                  ) 


train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/My Drive/Colab Notebooks/house/handwriting_shape/train',
        target_size=(24, 24),
        batch_size=32,
        class_mode='categorical') # trian data

test_datagen = ImageDataGenerator(rescale=1./255)  # nomalization

test_generator = test_datagen.flow_from_directory(
        '/content/gdrive/My Drive/Colab Notebooks/house/handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=32,
        class_mode='categorical') #test data
   
from keras.models import Sequential, Model
from keras import layers
from keras import regularizers
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Input, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam

classes = 4
filter_size = 16  # 필터 개수
kernel_size = (3,3) # 필터 사이즈
pool_size = (2,2)
input_shape = (24,24,3)
def getModel(input_shape):
  
  def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    
    conv_name_base = 'res' + str(stage) + block +  '_branch'
    bn_name_base = 'bn' + str(stage) + block +  '_branch'
    
    x = Conv2D(filters1, (1,1), name = conv_name_base +'2a')(input_tensor)
    x = BatchNormalization(name = bn_name_base+'2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,padding='same',name = conv_name_base+'2b')(x)
    x = BatchNormalization(name = bn_name_base +'2b')(x)
    x = Activation('relu')(x)
    
    
    x = Conv2D(filters3,(1,1),name = conv_name_base+'2c')(x)
    x = BatchNormalization(name = bn_name_base +'2c')(x)
    
    x = layers.add([x,input_tensor])
    x = Activation('relu')(x)
    
    return x
  
  def conv_block(input_tensor,kernel_size, filters,stage, block,strides=(2,2)):
    filters1, filters2 , filters3 = filters
    conv_name_base = 'res' + str(stage)+block+'_branch'
    bn_name_base = 'bn' +str(stage) +block+'_branch'
    
    x = Conv2D(filters1, (1,1),strides = strides, name = conv_name_base +'2a')(input_tensor)
    x = BatchNormalization(name = bn_name_base +'2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters2, kernel_size,padding = 'same', name = conv_name_base +'2b')(x)
    x = BatchNormalization(name = bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters3,(1,1), name = conv_name_base +'2c')(x)
    x = BatchNormalization(name = bn_name_base+'2c')(x)
    
    shortcut = Conv2D(filters3, (1,1), strides = strides, name = conv_name_base+'1')(input_tensor)
    shortcut = BatchNormalization(name = bn_name_base+'1')(shortcut)
    
    x = layers.add([x,shortcut])
    x = Activation('relu')(x)
    
    return x
  
  i = Input(shape = input_shape)
  x = Conv2D(64, (7,7),strides = (2,2), padding = 'same', name = 'conv1')(i)
  x = BatchNormalization(name = 'bn_conv1')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D((3,3), strides =(2,2))(x)
 
  x = conv_block(x,3,[64,64,256],stage = 2 , block = 'a', strides =(1,1))
  x = identity_block(x,3,[64,64,256], stage = 2, block = 'b')
  x = identity_block(x,3,[64,64,256], stage = 2, block = 'c')
  x = conv_block(x,3,[128,128,512], stage = 3, block = 'a')
  x = identity_block(x,3,[128,128,512], stage = 3, block = 'b')
  x = identity_block(x,3,[128,128,512], stage = 3, block = 'c')
  x = identity_block(x,3,[128,128,512], stage = 3, block = 'd')
  
  x = AveragePooling2D((2,2), name = 'avg_pool')(x)
  
  x = Flatten()(x)
  x = Dense(64,input_dim= 64, kernel_regularizer = regularizers.l2(0.01))(x)
  x = Dense(classes,activation = "softmax", name = "softmax")(x)
  
  
  return Model(i,x, name = 'Resnet')
        

m = getModel(input_shape)


lr = 0.00123  # best learning rate from random search

# 모델 엮기
m.compile(loss='categorical_crossentropy', optimizer= Adam(lr), metrics=['accuracy'])

# 모델 학습시키기
history = m.fit_generator(
         train_generator,
        steps_per_epoch = 100,
        epochs=200, validation_data = test_generator, validation_steps = 4
       )

# 모델 평가하기
print("-- Evaluate --")

scores = m.evaluate_generator(
            test_generator, 
            steps = 4)

print("%s: %.2f%%" %(m.metrics_names[1], scores[1]*100))
  # 훈련의 정확도, 손실 그래프 그리기
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss =  history.history['loss']
val_loss =  history.history['val_loss']
epochs = range(1,len(acc)+1)
plt.plot(epochs,acc,'bo',label = 'Training acc')
plt.plot(epochs,val_acc,'b',label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo',label = 'Training loss')
plt.plot(epochs,val_loss,'b',label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


m.save('m.h5') # 학습 모델 저장

In [0]:
from keras.models import load_model
m = load_model('m.h5')  #학습 모델 불러오기
Test_datagen = ImageDataGenerator(rescale=1./255) 

Test_generator = Test_datagen.flow_from_directory(
        '/content/gdrive/My Drive/Colab Notebooks/house/handwriting_shape/Test',
        target_size=(24, 24),    
        batch_size=32,
        class_mode='categorical') # 실제로 찍은 테스트 데이터 불러오기
scores = m.evaluate_generator(
            Test_generator, 
            steps = 4)
print("%s: %.2f%%" %(m.metrics_names[1], scores[1]*100)) # 실제 데이터 accuracy

In [0]:
from keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

img_width, img_height  =24, 24

validation_datagen = ImageDataGenerator(
      rescale=1./255,
      )

# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
      '/content/gdrive/My Drive/Colab Notebooks/house/handwriting_shape/Test',#경로 validation으로 수정하기
        target_size=(img_width, img_height),
        batch_size=10,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = m.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
right = np.where(predicted_classes == ground_truth)[0] 
# Show the errors




  
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = image.load_img('{}/{}'.format('/content/gdrive/My Drive/Colab Notebooks/house/handwriting_shape/Test',fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

        
for i in range(len(right)):
    pred_class_right = np.argmax(predictions[right[i]])
    pred_label_right = idx2label[pred_class_right]
     
    title_right = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[right[i]].split('/')[0],
        pred_label_right,
        predictions[right[i]][pred_class_right])
     
    original_right = image.load_img('{}/{}'.format('/content/gdrive/My Drive/Colab Notebooks/house/handwriting_shape/Test',fnames[right[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title_right)
    plt.imshow(original_right)
    plt.show()
    
cm = confusion_matrix(ground_truth, predicted_classes)
cm
TruePositive = np.diag(cm)
print("TP is" + str(TruePositive) )

FalsePositive = []
for i in range(4):
    FalsePositive.append(sum(cm[:,i]) - cm[i,i])

print("FP is" + str(FalsePositive) )
        
FalseNegative = []
for i in range(4):
    FalseNegative.append(sum(cm[i,:]) - cm[i,i])
    
print("FN is" + str(FalseNegative) )

TrueNegative = []
for i in range(4):
    temp = np.delete(cm, i, 0)   # delete ith row
    temp = np.delete(temp, i, 1)  # delete ith column
    TrueNegative.append(sum(sum(temp)))
   
print("TN is" + str(TrueNegative) )   

target_names = ['Glass', 'Metal','Paper','Plastic']
print(classification_report(ground_truth, predicted_classes, target_names=target_names))